# Introduction

This notebook shows how to predict the image classes with the use of TensorFlow.

It uses a simple home-made neural network model.

The description of the model is added with the explanation on how to compute the output shapes and number of parameters for the different layers. 
It can be used by someone intersted on how the layers are actually populated with neurons.

If you find it useful, please feel free to upvote it !

Thanks to Praveen for his notebook https://www.kaggle.com/prvnkmr/ranzcr-tf-baseline-lb-0-908, that helped me with to build the input datasets.

The notebook is divided into :


1) Load the csv

2) A small exploration

3) Create the train, validation and test datasets

4) Create the model

5) Train the model

6) Make previsions on the test data

7) Submit the model



# 0) Import modules

In [ ]:
import tensorflow as tf

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import seaborn as sns

from PIL import Image

import numpy as np
import pandas as pd
import os

# 1) Load csv

In [ ]:

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


#for dirname, _, filenames in os.walk('/kaggle/input'):
#    print(dirname)
#    for filename in filenames:
#        if dirname == '/kaggle/input/ranzcr-clip-catheter-line-classification':
#            print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# useful paths
catherer_path = '/kaggle/input/ranzcr-clip-catheter-line-classification'
train_path = os.path.join(catherer_path,'train')
test_path = os.path.join(catherer_path,'test')

In [ ]:
# Load train.csv data
train_csv_path = os.path.join(catherer_path,'train.csv')
train_df = pd.read_csv(train_csv_path)

# classes to predict
classes = [col for col in train_df.columns if col not in ['StudyInstanceUID','PatientID']]
print(classes)

# add the .jpg to the file names in the dataset
train_df['path_name'] = train_df['StudyInstanceUID'].apply(lambda x:os.path.join(train_path,x+'.jpg'))
print("Example of a path name : {}".format(train_df['path_name'][0]))
# shape of the train data
print("\nShape of train dataframe : {}\n".format(train_df.shape))
print("check for null values :")
print(train_df.isnull().sum())

In [ ]:
train_df.head()

In [ ]:
# Load test.csv data
test_csv_path = os.path.join(catherer_path,'sample_submission.csv')
test_df = pd.read_csv(test_csv_path)

# add the .jpg to the file names in the test dataset
test_df['path_name'] = test_df['StudyInstanceUID'].apply(lambda x:os.path.join(test_path,x+'.jpg'))
print("\nShape of test dataframe : {}\n".format(test_df.shape))
print("check for null values :")
print(test_df.isnull().sum())
#test_df.head()

In [ ]:
# Print one image
im_path = train_df['path_name'].iloc[0]
im_example = Image.open(im_path)
print("Image size = {}".format(im_example.size))
plt.figure(figsize=(12,8))
plt.imshow(im_example,cmap='Greys')

In [ ]:
# print the same image with less pixels
# We see that on a 256 X 256 pixels, some details are still visible
dim1 = 256
dim2 = 256
im_example_red = im_example.resize((dim1,dim2))
plt.figure(figsize=(12,8))
plt.imshow(im_example_red,cmap='Greys')

# 2) Exploration

In [ ]:
# True if you want to do the exploration steps, False otherwise
exploration = False 

In [ ]:
# Size of the images in the train dataset
# It can take a few minutes
if exploration:
    dim1 = []
    dim2 = []
    counter = 0
    for image_filename in os.listdir(train_path):
        counter+=1
        if np.mod(counter,1000) == 0:
            print("counter : {}".format(counter))
        img = Image.open(os.path.join(train_path,image_filename))
        d1,d2 = img.size
        dim1.append(d1)
        dim2.append(d2)
    sns.jointplot(dim1,dim2)
    print("Dimension 1 : {}".format(np.mean(dim1)))
    print("Dimension 2 : {}".format(np.mean(dim2)))

In [ ]:
# Number of images for each class
# We see that the ETT abnormal, NGT abnormal and NGT borderline classes have few images
if exploration:
    plt.figure(figsize=(10,6))
    graph = sns.barplot(x=classes,y=train_df[classes].sum())
    graph.set_xticklabels(graph.get_xticklabels(), rotation=90)


# 3) Create the train, validation and test data sets

In [ ]:
# Some parameters

# image size
im_width= 256
im_height = 256
# batch size
batch_size = 32

#steps_per_epoch = len(X_train) // batch_size
#print(steps_per_epoch)
#print(X_valid.shape)

In [ ]:
# Limit the number of train samples if you want to accelerate the training
# Can be used to test your model - see there are no bugs

lim = True
if lim:
    red_train_df = train_df.sample(frac=0.1)
else:
    red_train_df = train_df.copy()
print(red_train_df.shape)

In [ ]:
# copy the lines for the classes with few images and add them to the dataframe so that it is less imbalanced
# Note that an image can belong to multiple class, so that we also increase the number of images for the 
# classes with a lot of images

# minimal number of samples per class
n_min = 100
count_classes = red_train_df[classes].sum()
ext_train_df = [red_train_df]
for pred_class in classes:
    if count_classes[pred_class] < n_min:
        new_df = red_train_df[red_train_df[pred_class]==1].sample(n_min,replace=True)
        ext_train_df.append(new_df)
        
ext_train_df = pd.concat(ext_train_df)
print(ext_train_df.shape)

In [ ]:
# Show the number of images per class after adding new ones
plt.figure(figsize=(10,6))
graph = sns.barplot(x=classes,y=ext_train_df[classes].sum())
graph.set_xticklabels(graph.get_xticklabels(), rotation=90)

In [ ]:
# Split the train dataset into a train and a validation datasets
X_train, X_valid = train_test_split(ext_train_df,test_size=0.2,shuffle=True)
print(X_train.shape)
print(X_valid.shape)

In [ ]:
# Create datasets from the dataframes for the train, validation and test data
# For train and validation data, a slice is made up of the image path name and its labels
# For the test data, a slice is the image path name

Train_df = tf.data.Dataset.from_tensor_slices((X_train['path_name'].values, X_train[classes].values))

Valid_df = tf.data.Dataset.from_tensor_slices((X_valid['path_name'].values, X_valid[classes].values))

Test_df = tf.data.Dataset.from_tensor_slices((test_df['path_name'].values))

In [ ]:
# Show a slice
for path, label in Train_df.take(5):
    print ('Path: {}, Label: {}'.format(path, label))

In [ ]:
def process_data_train(image_path, label):
    # returns an image (type EagerTensor) and its labels
    # decode_jpeg : if channels = 3, same values on 3 planes, here I chose channel = 1
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=1)
    img = tf.image.random_brightness(img, 0.3)
    img = tf.image.random_flip_left_right(img)
    img = tf.image.resize(img, [im_height,im_width])
    
    return img, label

In [ ]:
def process_data_valid(image_path, label):
    # No image modification for the vaidation data
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=1) # !! idem
    img = tf.image.resize(img, [im_height,im_width])
    
    return img, label

In [ ]:
def process_data_test(image_path):
    # No labels for the test data
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=1) # !! idem
    img = tf.image.resize(img, [im_height,im_width])
    
    return img

In [ ]:
# For each slice, replace the path name with the image data
Train_df = Train_df.map(process_data_train, num_parallel_calls=tf.data.experimental.AUTOTUNE)
Valid_df = Valid_df.map(process_data_valid, num_parallel_calls=tf.data.experimental.AUTOTUNE)
Test_df = Test_df.map(process_data_test, num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [ ]:
# Show a slice
for image, label in Train_df.take(1):
    print ('Image: {}, Label: {}'.format(tf.reshape(image,(256,256)), label))

In [ ]:
def configure_for_performance(ds, batch_size = 32):
    
    ds = ds.cache('/kaggle/dump.tfcache') 
    #ds = ds.repeat()
    ds = ds.shuffle(buffer_size=1024)
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    
    return ds

train_ds_batch = configure_for_performance(Train_df)
valid_ds_batch = Valid_df.batch(32*2)
test_ds_batch = Test_df.batch(32*2)

# 4) Create the model

In [ ]:
# Create the model
# For the selection of the parameters of the convolutional layers, you can see on :
#https://stats.stackexchange.com/questions/148139/rules-for-selecting-convolutional-neural-network-hyperparameters

# Add one dimension to the image size, needed for Tensorflow
image_shape = (im_width,im_height,1) 
# number of classes
n_classes = len(classes)
print("There are {} classes to predict".format(n_classes))

# model
model = Sequential()
#model.add(tf.keras.layers.Input(shape=image_shape))
#model.add(tf.keras.layers.experimental.preprocessing.RandomRotation(0.05, interpolation='nearest'))

# Convolutional layers to filter the data and MaxPooling layers to reduce the model size
model.add(Conv2D(filters=32, kernel_size=(3,3),input_shape=image_shape, activation='relu',padding='valid'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3,3),input_shape=image_shape, activation='relu',padding='valid'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3,3),input_shape=image_shape, activation='relu',padding='valid'))
model.add(Conv2D(filters=64, kernel_size=(3,3),input_shape=image_shape, activation='relu',padding='valid'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flat layer
model.add(Flatten())

# Dense layer with 24 neurons
model.add(Dense(240, activation='relu'))

# Dense layer with 24 neurons
#model.add(Dense(30, activation='relu'))

# Drop out layer to reduce overfitting.
# 50 % of the neurons are randomly deactivated during training
model.add(Dropout(0.3))

# Dense class, activation = sigmoid because for each field to predict
# we have a binary choice
model.add(Dense(n_classes, activation='sigmoid'))

# compile the model
adam_opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(loss='binary_crossentropy',
              optimizer=adam_opt,
              metrics=[tf.keras.metrics.AUC(multi_label=True)]
             )

In [ ]:
# Summary of the model
# 
#  1 convolutional layer :
#      output shape = 254 X 254 X 32 
#                     254 X 254 because input shape = 256 X 256 and we have a kernel size = (3,3)
#                         with padding  = 'valid'
#                     32 because there are 32 filters  
#      param = 320 : kernel_size = (3,3) => 9 parameters by filter. Plus 1 bias => 10 parameters for a filter. 32 filters
#                    => 32 X 10 = 320
#
# 1 MaxPooling (2,2):
#      output shape = (127,127,32) because input size is divided by 2 in width and height, with still 32 filters
#
# 1 convolutional layer :
#      output shape = 125 X 125 X 64 (same explanation as before and 64 filters) 
#      param = 18496 : 64 filters in output, each applied on an input cell of 3 X 3 X 32
#                      (in each of the 32 input filters, the is kernel size = (3,3))
#                      Plus 64 bias (one for each filter)
#                      => 18496 = 64 X 32 X 9 + 64
#
# 1 MaxPooling (2,2):
#      output shape = (62,62,64) because input size is divided by 2 in width and height, with still 64 filters
#
# 1 convolutional layer :
#      output shape = 60 X 60 X 64 (same explanation as before and 64 filters) 
#      param = 36928 : 64 filters in output, each applied on an input cell of 3 X 3 X 64
#                      (in each of the 64 input filters, the kernel size = (3,3))
#                      Plus 64 bias (one for each filter)
#                      => 36928 = 64 X 64 X 9 + 64
#
# 1 convolutional layer :
#      output shape = 58 X 58 X 64 (same explanation as before and 64 filters) 
#      param = 36928 : 64 filters in output, each applied on an input cell of 3 X 3 X 64
#                      (in each of the 64 input filters, the kernel size = (3,3))
#                      Plus 64 bias (one for each filter)
#                      => 36928 = 64 X 64 X 9 + 64
#
# 1 MaxPooling (2,2):
#      output shape = (29,29,64) because input size is divided by 2 in width and height, with still 64 filters
# 
# 1 Flatten layer :
#      ouput shape = 53 824 = 29 X 29 X 64
#
# 1 Dense layer with 240 neurons
#      param = 12 918 000 : 240 neurons linked to 53 824 neurons, plus 240 bias
#               => 12 918 000 = 240 X 53 824 + 240
#
# 1 Drop out layer : 50 % of the neurons in the previous Dense layer are not selected in training
#
# 1 Dense layer :
#      output shape = 11 because 11 classes to predict
#      param = 2651 = 11 X 240 + 11 
# 
# We see that 12 000 000 parameters out of 13 000 000 come from the first Dense layer alone

model.summary()

In [ ]:
# Early stopping
# If 2 (parameter patience) epochs are run with a decrease in the validation loss, 
# stop the training
early_stop = EarlyStopping(monitor='val_loss',patience=2)

In [ ]:
# Checkpoint to save the "best" model parameters
checkpoint = ModelCheckpoint(
    'best_model.hdf5', monitor='val_loss', save_best_only=True,
    save_weights_only=False, mode='auto'
)

# 5) Train the model

In [ ]:
# Train the model
results = model.fit(train_ds_batch,#train_generator,
                    epochs=1,
                    batch_size=32,
                    validation_data=valid_ds_batch,#valid_generator,
                    callbacks=[early_stop, checkpoint],
                    verbose=True,
   # steps_per_epoch=steps_per_epoch
                   )

In [ ]:
# History of the loss throughout the epochs
losses = pd.DataFrame(model.history.history)
print(model.metrics_names)
losses[['loss','val_loss','auc','val_auc']].plot()

In [ ]:
# Load the best weights
model.load_weights('best_model.hdf5') 

In [ ]:
# save the model in case you want to reuse it
#model.save('my_model')
# load model
#saved_model = load_model('my_model')

# 6) Make previsions on the test data

In [ ]:
# Predict on the test data
pred_probabilities = model.predict(test_ds_batch)

In [ ]:
# Some info
print(pred_probabilities.shape)
print(pred_probabilities[0])
print(pred_probabilities[100])

# 7) Submit the model

In [ ]:
# Create a dataframe with the predicted probabilities for the test images
pred_df = pd.DataFrame(columns=classes,data=pred_probabilities, index=test_df.index)
pred_df = pd.concat([test_df['StudyInstanceUID'],pred_df],axis=1)
pred_df

In [ ]:
# Create the csv
pred_df.to_csv('submission.csv',index=False)